In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Calibri"

# Grain Data

In [2]:
g_data=pd.read_excel("KFB1H_Light&Heavy.xlsx",skiprows=0,sheet_name=1)

R_std=[0.05679513,1,0.70993915,8.37525355]

# e84=g_data[g_data['dSr84']> 0] # grains that are excess in 84
really_e84=g_data[g_data['dSr84']-2*g_data['dSr84 Sig']> 0] # filtering excess in 84Sr within 2 sigma error

e=len(really_e84)
Sr_really_e84=np.zeros([e,4])
err_Sr_really_e84=np.zeros([e,4])
for i in range(0,e):
    Sr_really_e84[i]=[float(x) for x in really_e84.iloc[i][13:21:2]]   #array of the 5 84Sr excess grains Sr delta values  
    err_Sr_really_e84[i]=[float(x) for x in really_e84.iloc[i][14:22:2]]


Sr1_really_e84=np.delete(Sr_really_e84,1,1)#delete the 86 column
R_Sr_really_e84=(Sr_really_e84/1000+1)*R_std  #ratio values of the grains
R_Sr1_really_e84=np.delete(R_Sr_really_e84,1,1) #delete the 86 column


# Mixing Model

In [3]:
#array with model file names. We used six model files. 
Star_model = ['Ritter_0p02_15Msun_Lorenzo.dat', 'Ritter_0p02_20Msun_Lorenzo.dat', 'Ritter_0p02_25Msun_Lorenzo.dat',
              'Rauscher_15Msun_Lorenzo.dat', 'Rauscher_20Msun_Lorenzo.dat', 'Rauscher_25Msun_Lorenzo.dat']

#stellar zones 
Zone_names=['H','C','N','He','O','Ne','Si','S','Ni']

Grains_mixedlayer1 = np.zeros([e,len(Star_model)],dtype=int)
Grains_mixedlayer2 = np.zeros([e,len(Star_model)],dtype=int)
Grains_k1mix = np.zeros([e,len(Star_model)])
Grains_k2mix = np.zeros([e,len(Star_model)])
Grains_mixedlayer1Zone = np.empty(shape=[e,len(Star_model)],dtype=object)
Grains_mixedlayer2Zone = np.empty(shape=[e,len(Star_model)],dtype=object)
delta_res = np.zeros([e,len(Star_model),4])

for S in range(len(Star_model)):
    data = pd.read_table(Star_model[S],delimiter='\s+').drop([0,1])
    Zone_arr = np.array((data["H-1"],data["C-12"],data["N-14"],data["He-4"],data["O-16"],data["Ne-20"],data["Si-28"],data["S-32"],data["Ni-56"])).T
    
    for z in range(len(Zone_arr)): # Finding beginning of O/Si zone
        Z1 = np.where(Zone_arr[z] == sorted(Zone_arr[z])[-1])[0][0]
        Z2 = np.where(Zone_arr[z] == sorted(Zone_arr[z])[-2])[0][0]
        if ((Z1 == 4 and Z2 == 6) or (Z1 == 6 and Z2 == 4)): break
        else: z = 0 
        
    data = data[z:] # Setting data to start at O/Si zone
    print("Running Model:",Star_model[S],"O/Si base:",z,len(data)+z)
    
    R_std=[0.05679513,0.70993915,8.37525355]
        
    data_Sr84 = np.array(data["Sr-84"]+data["Y-84"]+data["Zr-84"])/84
    data_Sr86 = np.array(data["Sr-86"]+data["Rb-86"]+data["Y-86"]+data["Zr-86"])/86
    data_Sr87 = np.array(data["Sr-87"]+data["Y-87"]+data["Zr-87"])/87
    data_Sr88 = np.array(data["Sr-88"]+data["Rb-88"]+data["Y-88"]+data["Kr-88"]+data["Zr-88"])/88
    C_total=np.array(data["C-12"]/12+data["C-13"]/13+data["C-11"]/11+data["C-14"]/14+data["C-15"]/15)
    O_total=np.array(data["O-16"]/16+data["O-17"]/17+data["O-18"]/18+data["O-13"]/13+data["O-14"]/14+data["O-15"]/15+data["O-19"]/19)
        
    A_Sr = np.array([data_Sr84,data_Sr86,data_Sr87,data_Sr88]).T
    R_Sr = (A_Sr.T/A_Sr[:,1]).T          # Ratio to 86 Sr
    D_Sr = (R_Sr/np.insert(R_std,1,1)-1)*1000   # delta values    
    R_star1 = np.delete(R_Sr,1,1)
    star1=(R_star1/R_std-1)*1000    # delta values    
    
    for grainNow in range(0,e):

        target=R_Sr1_really_e84[grainNow]   #ratio of target grain

        minorP=A_Sr[R_Sr[:,0]<target[0]] #all star layers with 84 ratio below target grain
        majorP=A_Sr[R_Sr[:,0]>target[0]] #all star layers with 84 ratio below target grain
        
        C_minorP=C_total[R_Sr[:,0]<target[0]] #C and O values for those layers
        O_minorP=O_total[R_Sr[:,0]<target[0]]
        C_majorP=C_total[R_Sr[:,0]>target[0]]
        O_majorP=O_total[R_Sr[:,0]>target[0]]
        
        residual_mix=np.zeros([len(majorP),len(minorP)]) #initiation
        k1_mix=np.zeros([len(majorP),len(minorP)])
        k2_mix=np.zeros([len(majorP),len(minorP)])
        C_O=np.zeros([len(majorP),len(minorP)])
    #     C12_13=np.zeros([len(majorP),len(minorP)])
    
        for i in range(len(majorP)):
            for j in range(len(minorP)):
                k1_mix[i][j] = (minorP[j][0] - target[0]*minorP[j][1])/(target[0]*(majorP[i][1] - minorP[j][1])-majorP[i][0] + minorP[j][0])
                k2_mix[i][j] = 1-k1_mix[i][j] # k2=1-k1

                model_A = k2_mix[i][j]*minorP[j]+k1_mix[i][j]*majorP[i] # model of entire layer with k1 and k2
                model = np.delete((model_A/ model_A[1]),1) # converting to ratio              
                residual_mix[i,j] = sum((target-model)**2) #difference of model with the grain
                C_O[i][j] = (k1_mix[i][j]*C_majorP[i]+k2_mix[i][j]*C_minorP[j])/(k1_mix[i][j]*O_majorP[i]+k2_mix[i][j]*O_minorP[j])
                
        indx = (residual_mix>0) * (C_O>1) #index with the filter of C/O>1; comment the *(C_O>1) part out for non C/O constraint
        if(np.sum(indx)>0): 
            minRes = np.where(residual_mix == min(residual_mix[indx])) #finding the least difference
        
            # Saving layer values
            layer1=minRes[0][0]; Grains_mixedlayer1[grainNow,S] = np.where(A_Sr==majorP[layer1])[0][0]+z+2 # Saving Layer 1
            layer2=minRes[1][0]; Grains_mixedlayer2[grainNow,S] = np.where(A_Sr==minorP[layer2])[0][0]+z+2 # Saving Layer 2

            # Saving mixing coefficients
            Grains_k1mix[grainNow,S] = k1_mix[layer1][layer2]
            Grains_k2mix[grainNow,S] = k2_mix[layer1][layer2]
            
            print(Grains_k1mix[grainNow,S],":",Grains_mixedlayer1[grainNow,S],"+",Grains_k2mix[grainNow,S],":",Grains_mixedlayer2[grainNow,S])

            res = k1_mix[layer1][layer2]*majorP[layer1]+k2_mix[layer1][layer2]*minorP[layer2] #resultant model layer 
            delta_res[grainNow,S] = ((res/res[1])/np.insert(R_std,1,1)-1)*1000 # transferring back to delta
            
            ZI1=(np.where(Zone_arr[Grains_mixedlayer1[grainNow,S]]==sorted(Zone_arr[Grains_mixedlayer1[grainNow,S]])[-1]))[0][0]
            ZI2=(np.where(Zone_arr[Grains_mixedlayer1[grainNow,S]]==sorted(Zone_arr[Grains_mixedlayer1[grainNow,S]])[-2]))[0][0]
            L1='('+Zone_names[int(ZI1)]+'/'+Zone_names[int(ZI2)]+')'
            if(Zone_names[int(ZI1)]=="Ni" or Zone_names[int(ZI1)]=="Ni"): L1="(Ni)"
            if (L1=="(He/H)"): L1="(env.)"
            if(L1=='(Si/O)'): L1='(O/Si)'
            Grains_mixedlayer1Zone[grainNow,S] = L1
            
            ZI1=(np.where(Zone_arr[Grains_mixedlayer2[grainNow,S]]==sorted(Zone_arr[Grains_mixedlayer2[grainNow,S]])[-1]))[0][0]
            ZI2=(np.where(Zone_arr[Grains_mixedlayer2[grainNow,S]]==sorted(Zone_arr[Grains_mixedlayer2[grainNow,S]])[-2]))[0][0]
            L2='('+Zone_names[int(ZI1)]+'/'+Zone_names[int(ZI2)]+')'
            if(Zone_names[int(ZI1)]=="Ni" or Zone_names[int(ZI1)]=="Ni"): L2="(Ni)"
            if(L2=='(He/H)'): L2="(env.)"
            if(L2=='(Si/O)'): L2='(O/Si)'
            Grains_mixedlayer2Zone[grainNow,S] = L2
        
        else:
            print("No Mixing Sol. Grain:",grainNow)

Running Model: Ritter_0p02_15Msun_Lorenzo.dat O/Si base: 173 2158
0.003660191423676321 : 212 + 0.9963398085763236 : 1373
0.023399980016014232 : 208 + 0.9766000199839857 : 1372
0.041029653426547436 : 206 + 0.9589703465734526 : 1373
0.003785595770902441 : 214 + 0.9962144042290976 : 1373
0.004873387866716476 : 212 + 0.9951266121332836 : 1372
Running Model: Ritter_0p02_20Msun_Lorenzo.dat O/Si base: 0 2088
0.003758565420713572 : 15 + 0.9962414345792864 : 1241
0.012610838139440736 : 14 + 0.9873891618605593 : 1241
0.0022771638653858893 : 21 + 0.9977228361346141 : 1241
0.0006281141106204777 : 29 + 0.9993718858893795 : 1241
0.012832665428830131 : 13 + 0.9871673345711699 : 1240
Running Model: Ritter_0p02_25Msun_Lorenzo.dat O/Si base: 0 1669
No Mixing Sol. Grain: 0
No Mixing Sol. Grain: 1
No Mixing Sol. Grain: 2
No Mixing Sol. Grain: 3
No Mixing Sol. Grain: 4
Running Model: Rauscher_15Msun_Lorenzo.dat O/Si base: 46 653
0.010182475818218396 : 58 + 0.9898175241817816 : 262
0.04084802352056759 : 55 